In [ ]:
import os, sys, math, scipy
import pandas as pd
import numpy as np

sys.path.append("/home/rty10/Documents/scripts/analysis/")
import dna_analysis
import circle_data_compile as opt_compile

path = os.getcwd()

#optpath = path + '/OPT_DATA'
#optpath  = path + '/circular'
#freepath = path + '/free_collection'
#bxxpath  = path + '/bxx_circular'

forcefieldpath = "/home/rty10/Documents/forcefields"

## Compile: all opt global data to single .csv

In [ ]:
circles = sorted([ i.split('.')[0] for i in os.listdir(optpath) if i.endswith('.log') ])

NCIRC=336
df = pd.DataFrame()

for i in range(len(circles)):
    name = circles[i]
    
    df.at[i,'length']   = NCIRC
    df.at[i,'state']    = name.split('_')[0].split('-')[1]
    df.at[i,'scdna']    = int(name.split('_')[0].split('-')[1][2:]) - 32
    df.at[i,'sequence'] = 'zech-000'
    if "elcirc" in name:
        df.at[i,'config']      = 'circ'
        df.at[i,'opt_electro'] = 'y'
    else:
        df.at[i,'config']      = 'circ'
        df.at[i,'opt_electro'] = 'n'
    
    if 'IdealDNA' in name:
        df.at[i,'forcefield'] = 'ideal'
        df.at[i,'ff_type']    = 'dim'
    else:
        df.at[i,'forcefield'] = name.split('_')[1].split('-')[0]
        df.at[i,'ff_type']    = name.split('_')[1].split('-')[1]
    
    df = opt_compile.log_file_data(optpath+'/'+name+'.log', i, df)
    df = opt_compile.refframe_file_data(optpath+'/'+name+'.dat', i, df, NCIRC)
    df = opt_compile.topology_file_data(optpath+'/topo_'+name+'.txt', i, df)        
    del name
    
df['eopt_bend']=df.eopt_tilt+df.eopt_roll
df[['length','lk', 'scdna']]=df[['length','lk','scdna']].astype(int)
df = df.sort_values(by=['state','config','forcefield'])
df = df.reset_index(drop=True)

df.to_csv(datapath+"/zech336_circ_dataset")
del df
del circles

In [ ]:
df = pd.read_csv(datapath+"/zech336_circ_dataset", index_col=0)

df.head(10)

In [ ]:
df.dtypes
#del df

## Compile: all opt structural data to single .csv

In [ ]:
circles = sorted([ i.split('.')[0] for i in os.listdir(path+"/OPT_DATA") if i.endswith('.par') ])

NCIRC = 336

for theta in ['tilt','roll','twist','shift','slide','rise']:
    
    df = pd.DataFrame(columns=[i for i in range(1, NCIRC+1)])
    for i in range(len(circles)):
        name = circles[i]

        df.at[i,'length']   = NCIRC
        df.at[i,'state']    = name.split('_')[0].split('-')[1]
        df.at[i,'scdna']    = int(name.split('_')[0].split('-')[1][2:]) - 32

        if "elcirc" in name:
            df.at[i,'config']      = 'circ'
            df.at[i,'opt_electro'] = 'y'
            if len(name.split('_')[2].split('-')) == 2:
                df.at[i,'sequence'] = name.split('_')[2].split('-')[1]
            else:
                df.at[i,'sequence'] = 'n000'
        else:
            df.at[i,'config']      = 'circ'
            df.at[i,'opt_electro'] = 'n'
            df.at[i,'sequence'] = 'n000'

        if 'IdealDNA' in name:
            df.at[i,'forcefield'] = 'ideal'
            df.at[i,'ff_type']    = 'dim'
        else:
            df.at[i,'forcefield'] = name.split('_')[1].split('-')[0]
            df.at[i,'ff_type']    = name.split('_')[1].split('-')[1]
            
        pardf = dna_analysis.df_read_bpsteppars(path+"/OPT_DATA/"+name+'.par', bend=False)        
        
        pardf = pardf[theta].drop([0],axis=0).T
        
        for j in range(1, NCIRC+1):
            df.at[i, j] = pardf[j]
        
        del pardf, name
        
    df=df[['state','sequence','config','forcefield','ff_type']+[i for i in range(1, NCIRC+1)]]
    
    df.to_csv(path+"/datasets/zech336_seq-thread_par-"+theta+"_dataset")
    del df
del circles, NCIRC

In [ ]:
NCIRC = 336

df      = pd.DataFrame(columns=[i for i in range(1, NCIRC+1)])
circles = sorted([ i.split('.')[0] for i in os.listdir(path+"/OPT_DATA") if i.endswith('.log') ])
dataset = pd.read_csv(path+"/datasets/zech336_circ-seq-thread_dataset")

for i in range(len(circles)):
    name = circles[i]

    df.at[i,'length']   = NCIRC
    df.at[i,'state']    = name.split('_')[0].split('-')[1]
    df.at[i,'scdna']    = int(name.split('_')[0].split('-')[1][2:]) - 32

    if "elcirc" in name:
        df.at[i,'config']      = 'circ'
        df.at[i,'opt_electro'] = 'y'
        if len(name.split('_')[2].split('-')) == 2:
            df.at[i,'sequence'] = name.split('_')[2].split('-')[1]
        else:
            df.at[i,'sequence'] = 'n000'
    else:
        df.at[i,'config']      = 'circ'
        df.at[i,'opt_electro'] = 'n'
        df.at[i,'sequence'] = 'n000'

    if 'IdealDNA' in name:
        df.at[i,'forcefield'] = 'ideal'
        df.at[i,'ff_type']    = 'dim'
    else:
        df.at[i,'forcefield'] = name.split('_')[1].split('-')[0]
        df.at[i,'ff_type']    = name.split('_')[1].split('-')[1]
    
    pardf = dna_analysis.df_read_bpsteppars(path+"/OPT_DATA/"+name+'.par', bend=False)
    
    if 'tet' in name:
        pardf = dna_analysis._par_tetrameric(pardf)
    
    if 'ideal' in name or 'Ideal' in name:
        FORCEFIELD="IdealDNA"
    elif "olson" in name or 'Olson' in name:
        FORCEFIELD="Olson1998"
    else:
        FORCEFIELD=name.split('_')[1]
    reststatedf = dna_analysis.df_read_intrinsic_state(forcefieldpath, FORCEFIELD)
    forcecondf  = dna_analysis.df_read_force_constants(forcefieldpath, FORCEFIELD)
    del FORCEFIELD

    if 'tet' in name:
        pardf = dna_analysis._par_energetic_analysis(pardf, reststatedf, forcecondf, tet=True)
    else:
        pardf = dna_analysis._par_energetic_analysis(pardf, reststatedf, forcecondf)
        
    for j in range(1, NCIRC+1):
        df.at[i, j] = pardf.at[j, 'energy']
    df.at[i, 'sum'] = df.loc[i][0:NCIRC].sum()
    df.at[i, 'eopt']=dataset[(dataset['state']     == df.at[i, 'state'])
                           &(dataset['sequence']   == df.at[i, 'sequence'])
                           &(dataset['forcefield'] == df.at[i, 'forcefield'])
                           &(dataset['ff_type']    == df.at[i, 'ff_type'])
                           &(dataset['config']     == df.at[i, 'config'])
                           &(dataset['opt_electro']== df.at[i, 'opt_electro'])].eopt.values[0]
    del name
    del pardf
    del reststatedf, forcecondf

df.to_csv(path+"/datasets/zech336_circ-seq-threat-energy_dataset")
del df

del dataset, circles, NCIRC

## Compile: all global optimized data from single-bound opt collection

In [ ]:
circles = sorted([ i.split('.')[0] for i in os.listdir(bxxpath) if i.endswith('.log') ])

NCIRC=336

df = pd.DataFrame()

for i in range(len(circles)):
    name = circles[i]
    
    df.at[i,'length']   = NCIRC
    df.at[i,'state']    = name.split('_')[0].split('-')[1]
    df.at[i,'scdna']    = int(name.split('_')[0].split('-')[1][2:]) - 32
    df.at[i,'sequence'] = int(name.split('_')[2].lstrip('seq'))
    
    df.at[i,'config']      = 'circ'
    df.at[i,'opt_electro'] = 'n'
    
    df.at[i,'forcefield'] = name.split('_')[3].split('-')[0]
    df.at[i,'ff_type']    = name.split('_')[3].split('-')[1]
    
    df = opt_compile.log_file_data(bxxpath+'/'+name+'.log', i, df)
    df = opt_compile.refframe_file_data(bxxpath+'/'+name+'.dat', i, df, NCIRC)
    df = opt_compile.topology_file_data(bxxpath+'/topo_'+name+'.txt', i, df)        
    del name
    
df['eopt_bend']=df.eopt_tilt+df.eopt_roll
df[['length','lk', 'scdna']]=df[['length','lk','scdna']].astype(int)
df = df.sort_values(by=['state','config','forcefield'])
df = df.reset_index(drop=True)

df.to_csv(datapath+"/zech336_bxx-circ_dataset")
del df
del circles

## Compile: all opt BXX structural data to single .csv

In [ ]:
circles = sorted([ i.split('.')[0] for i in os.listdir(bxxpath) if i.endswith('.par') ])

NCIRC = 336

for theta in ['tilt','roll','twist','shift','slide','rise']:
    
    df = pd.DataFrame(columns=[i for i in range(1, NCIRC+1)])
    for i in range(len(circles)):
        name = circles[i]

        df.at[i,'length']   = NCIRC
        df.at[i,'state']    = name.split('_')[0].split('-')[1]
        df.at[i,'scdna']    = int(name.split('_')[0].split('-')[1][2:]) - 32
        df.at[i,'sequence'] = int(name.split('_')[2].lstrip('seq'))
        
        df.at[i,'config']      = 'circ'
        df.at[i,'opt_electro'] = 'n'

        df.at[i,'forcefield']  = name.split('_')[3].split('-')[0]
        df.at[i,'ff_type']     = name.split('_')[3].split('-')[1]
        
        pardf = dna_analysis.df_read_bpsteppars(bxxpath+'/'+name+'.par', bend=False)
        pardf = pardf[theta].drop([0],axis=0).T
        
        for j in range(1, NCIRC+1):
            df.at[i, j] = pardf[j]
        
        del pardf, name
        
    df=df[['state','sequence','config','forcefield','ff_type']+[i for i in range(1, NCIRC+1)]]
    
    df.to_csv(datapath+"/zech336_bxx-par-"+theta+"_dataset")
    del df
del circles, NCIRC

In [ ]:
NCIRC = 336

df      = pd.DataFrame(columns=[i for i in range(1, NCIRC+1)])
circles = sorted([ i.split('.')[0] for i in os.listdir(bxxpath) if i.endswith('.log') ])
dataset = pd.read_csv(datapath+"/zech336_bxx-circ_dataset", index_col=0)

for i in range(len(circles)):
    name = circles[i]

    df.at[i,'length']   = NCIRC
    df.at[i,'state']    = name.split('_')[0].split('-')[1]
    df.at[i,'scdna']    = int(name.split('_')[0].split('-')[1][2:]) - 32
    df.at[i,'sequence'] = int(name.split('_')[2].lstrip('seq'))

    df.at[i,'config']      = 'circ'
    df.at[i,'opt_electro'] = 'n'

    df.at[i,'forcefield']  = name.split('_')[3].split('-')[0]
    df.at[i,'ff_type']     = name.split('_')[3].split('-')[1]
    
    pardf = dna_analysis.df_read_bpsteppars(bxxpath+'/'+name+'.par', bend=False)
    
    if 'tet' in name:
        pardf = dna_analysis._par_tetrameric(pardf)
    
    if 'ideal' in name or 'Ideal' in name:
        FORCEFIELD="IdealDNA"
    elif "olson" in name or 'Olson' in name:
        FORCEFIELD="Olson1998"
    else:
        FORCEFIELD=name.split('_')[3]
    reststatedf = dna_analysis.df_read_intrinsic_state(forcefieldpath, FORCEFIELD)
    forcecondf  = dna_analysis.df_read_force_constants(forcefieldpath, FORCEFIELD)
    del FORCEFIELD

    if 'tet' in name:
        pardf = dna_analysis._par_energetic_analysis(pardf, reststatedf, forcecondf, tet=True)
    else:
        pardf = dna_analysis._par_energetic_analysis(pardf, reststatedf, forcecondf)
        
    for j in range(1, NCIRC+1):
        df.at[i, j] = pardf.at[j, 'energy']
    df.at[i, 'sum'] = df.loc[i][0:NCIRC].sum()
    df.at[i, 'eopt']=dataset[(dataset['state']     == df.at[i, 'state'])
                           &(dataset['sequence']   == df.at[i, 'sequence'])
                           &(dataset['forcefield'] == df.at[i, 'forcefield'])
                           &(dataset['ff_type']    == df.at[i, 'ff_type'])
                           &(dataset['config']     == df.at[i, 'config'])
                           &(dataset['opt_electro']== df.at[i, 'opt_electro'])].eopt.values[0]
    del name
    del pardf
    del reststatedf, forcecondf

df.to_csv(datapath+"/zech336_bxx-circ-energy_dataset")
del df

del circles, NCIRC

## scratch

In [ ]:
optpath84  = path + '/OPT_DATA'

circles = sorted([ i.split('.')[0] for i in os.listdir(optpath84) if i.endswith('.log') ])

NCIRC=336
df = pd.DataFrame()

for i in range(len(circles)):
    name = circles[i]
    
    df.at[i,'length']   = NCIRC
    df.at[i,'state']    = name.split('_')[0].split('-')[1]
    df.at[i,'scdna']    = int(name.split('_')[0].split('-')[1][2:]) - 32
    
    if "elcirc" in name:
        df.at[i,'config']      = 'circ'
        df.at[i,'opt_electro'] = 'y'
        if len(name.split('_')[2].split('-')) == 2:
            df.at[i,'sequence'] = name.split('_')[2].split('-')[1]
        else:
            df.at[i,'sequence'] = 'n000'
    else:
        df.at[i,'config']      = 'circ'
        df.at[i,'opt_electro'] = 'n'
        df.at[i,'sequence'] = 'n000'
    
    if 'IdealDNA' in name:
        df.at[i,'forcefield'] = 'ideal'
        df.at[i,'ff_type']    = 'dim'
    else:
        df.at[i,'forcefield'] = name.split('_')[1].split('-')[0]
        df.at[i,'ff_type']    = name.split('_')[1].split('-')[1]
    
    df = opt_compile.log_file_data(optpath84+'/'+name+'.log', i, df)
    df = opt_compile.refframe_file_data(optpath84+'/'+name+'.dat', i, df, NCIRC)
    df = opt_compile.topology_file_data(optpath84+'/topo_'+name+'.txt', i, df)        
    del name
    
df['eopt_bend']=df.eopt_tilt+df.eopt_roll
df[['length','lk', 'scdna']]=df[['length','lk','scdna']].astype(int)
df = df.sort_values(by=['state','config','forcefield'])
df = df.reset_index(drop=True)

#df.to_csv(datapath+"/zech336_circ_dataset")
#del df
del circles

In [ ]:
df

In [ ]:
df[(df.forcefield=='young2022')&(df.scdna==-3)]

In [ ]:
df[(df.state=='st34')&(df.forcefield=="young2022")&(df.ff_type=="tet")]

In [ ]:
df[(df.state=='st35')&(df.forcefield=="young2022")&(df.ff_type=="tet")]

In [ ]:
df[['state','lk']]

In [ ]:
df.to_csv(path+"/datasets/zech336_circ-seq-thread_dataset")
del df